In [28]:
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
from sklearn.model_selection import train_test_split

In [40]:
loan = pd.read_csv("data/loan.csv", low_memory=False, nrows=10000)
loan = loan[loan["loan_status"] != "Current"]

data = loan[["annual_inc", "int_rate", "purpose", "loan_amnt"]]
data = pd.get_dummies(features)
data["profit"] = loan.apply(lambda x: ((x['total_pymnt'] - x['loan_amnt'])/x['loan_amnt']), axis = 1)
data["profit"] = data["profit"].round(2)

train_dataset = data.sample(frac=0.8,random_state=0)
test_dataset = data.drop(train_dataset.index)

#x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

In [44]:
train_stats = train_dataset.describe()
train_stats.pop("profit")
train_stats = train_stats.transpose()
train_stats

train_labels = train_dataset.pop('profit')
test_labels = test_dataset.pop('profit')

def norm(x):
      return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [47]:
def build_model():
    print(train_dataset.keys())
    model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
    ])

    optimizer = tf.train.RMSPropOptimizer(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

In [48]:
model = build_model()

Index(['annual_inc', 'int_rate', 'loan_amnt', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',
       'purpose_medical', 'purpose_moving', 'purpose_other',
       'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding'],
      dtype='object')


In [49]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 500

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................

In [63]:
pred = model.predict(normed_test_data).flatten()

In [69]:
print(pred)
test_labels = test_labels.values.tolist()

[0.14257239 0.08156215 0.09419264 ... 0.04490371 0.0626417  0.21986048]


In [70]:
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
!pip install scipy
from scipy.stats import kurtosis, skew


er = []
g = 0
for i in range(len(test_labels)):
    #print( "actual=", y_test[i], " observed=", pred[i])
    x = (test_labels[i] - pred[i]) **2
    er.append(x)
    g = g + x
    
x = 0
for i in range(len(er)):
    x = x + er[i]
print("er", sum(er))
print ("MSE", x / len(er))

v = np.var(er)
print ("variance", v)

print ("average of errors ", np.mean(er))

m = np.mean(test_labels)
print ("average of observed values", m)

y = 0
for i in range(len(test_labels)):
    y = y + ((test_labels[i] - m) ** 2)
    

#print ("total sum of squares", y)
#print ("ẗotal sum of residuals ", g)
#print ("r2 calculated", 1 - (g / y))
print("kurtosis", kurtosis(er))
print("skewness", skew(er))


er 159.82839431597273
MSE 0.08835179343060957
variance 0.047066067348969176
average of errors  0.08835179343060984
average of observed values 0.07042564953012716
kurtosis 25.665604114774194
skewness 4.383910211288911
